# Copy Congress Strategy - Risk and Capacity Analysis

This notebook analyzes risk characteristics, capacity constraints, and regulatory considerations.

## Objectives
1. Analyze strategy capacity and market impact
2. Evaluate regulatory and reputational risks
3. Test parameter sensitivity
4. Assess signal decay and information lag

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import warnings
warnings.filterwarnings('ignore')

from data_acquisition import CongressionalDataAcquisition
from signal_generator import SignalGenerator
from portfolio_constructor import PortfolioConstructor
from backtester import CongressBacktester

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print('Imports complete')

## 1. Load Data and Run Baseline

In [ ]:
# Load configuration
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Load data
data_acq = CongressionalDataAcquisition(config)
congressional_trades, prices, volumes, market_caps, volatility = data_acq.get_full_dataset()

print('Data loaded')

In [ ]:
# Run baseline strategy
signal_gen = SignalGenerator(config)
portfolio_constructor = PortfolioConstructor(config)
backtester = CongressBacktester(config)

rebalance_dates = pd.date_range(
    start=prices.index[0],
    end=prices.index[-1],
    freq=config['portfolio']['rebalance_frequency']
)

signals_history = signal_gen.generate_signals_timeseries(
    congressional_trades, prices, rebalance_dates.tolist()
)

portfolio_history = portfolio_constructor.generate_portfolio_timeseries(
    signals_history, volatility, rebalance_dates.tolist()
)

baseline_results = backtester.simulate_portfolio(portfolio_history, prices)
baseline_metrics = backtester.calculate_performance_metrics(baseline_results)

print('\nBaseline Performance:')
print(f"Sharpe Ratio: {baseline_metrics['sharpe_ratio']:.2f}")
print(f"Annualized Return: {baseline_metrics['annualized_return']:.2%}")
print(f"Max Drawdown: {baseline_metrics['max_drawdown']:.2%}")

## 2. Capacity Analysis

In [ ]:
# Estimate strategy capacity based on trading volumes
print('Strategy Capacity Analysis')
print('='*60)

# Calculate average daily volume for universe
avg_daily_volume = volumes.mean().mean()
print(f"Average daily volume: {avg_daily_volume:,.0f} shares")

# Estimate max portfolio size
# Rule of thumb: don't exceed 5% of average daily volume
volume_threshold = 0.05
avg_turnover = config['monitoring']['max_turnover'] / 52  # Weekly turnover

# Conservative capacity estimate
avg_price = prices.mean().mean()
capacity_estimate = (
    avg_daily_volume * avg_price * volume_threshold / avg_turnover
)

print(f"\nEstimated strategy capacity: ${capacity_estimate:,.0f}")
print(f"Current portfolio size: ${config['backtest']['initial_capital']:,.0f}")
print(f"Capacity utilization: {config['backtest']['initial_capital'] / capacity_estimate:.1%}")

# Market impact analysis
impact_estimate = config['backtest']['initial_capital'] / capacity_estimate
if impact_estimate < 0.1:
    print("\nMarket impact: LOW - Minimal price impact expected")
elif impact_estimate < 0.5:
    print("\nMarket impact: MEDIUM - Some price impact possible")
else:
    print("\nMarket impact: HIGH - Significant price impact likely")

In [ ]:
# Turnover analysis
print('\nTurnover Analysis:')
print('='*60)

# Calculate actual turnover
portfolio_dates = sorted(portfolio_history.keys())
turnovers = []

for i in range(1, len(portfolio_dates)):
    old_weights = portfolio_history[portfolio_dates[i-1]]
    new_weights = portfolio_history[portfolio_dates[i]]
    
    turnover = portfolio_constructor.calculate_portfolio_turnover(old_weights, new_weights)
    turnovers.append(turnover)

turnovers = pd.Series(turnovers)

print(f"Average turnover per rebalance: {turnovers.mean():.1%}")
print(f"Median turnover: {turnovers.median():.1%}")
print(f"Max turnover: {turnovers.max():.1%}")
print(f"\nEstimated annual turnover: {turnovers.mean() * 52:.1%}")

# Plot turnover distribution
plt.figure(figsize=(12, 6))
plt.hist(turnovers * 100, bins=30, edgecolor='black')
plt.axvline(x=turnovers.mean() * 100, color='red', linestyle='--', 
           label=f'Mean: {turnovers.mean():.1%}')
plt.xlabel('Turnover (%)')
plt.ylabel('Frequency')
plt.title('Portfolio Turnover Distribution')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Parameter Sensitivity Analysis

In [ ]:
# Test different lookback windows
print('Testing lookback window sensitivity...')

lookback_windows = [15, 30, 45, 60, 90]
lookback_results = []

for window in lookback_windows:
    print(f"Testing {window}-day lookback...")
    
    # Modify config
    test_config = config.copy()
    test_config['signal']['lookback_days'] = window
    
    # Run strategy
    test_signal_gen = SignalGenerator(test_config)
    test_signals = test_signal_gen.generate_signals_timeseries(
        congressional_trades, prices, rebalance_dates.tolist()
    )
    
    test_portfolio_constructor = PortfolioConstructor(test_config)
    test_portfolios = test_portfolio_constructor.generate_portfolio_timeseries(
        test_signals, volatility, rebalance_dates.tolist()
    )
    
    test_backtester = CongressBacktester(test_config)
    test_results = test_backtester.simulate_portfolio(test_portfolios, prices)
    test_metrics = test_backtester.calculate_performance_metrics(test_results)
    
    lookback_results.append({
        'lookback': window,
        'sharpe': test_metrics['sharpe_ratio'],
        'return': test_metrics['annualized_return'],
        'volatility': test_metrics['volatility'],
        'max_dd': test_metrics['max_drawdown']
    })

lookback_df = pd.DataFrame(lookback_results)
print('\nLookback Window Sensitivity:')
print(lookback_df.round(3))

In [ ]:
# Visualize lookback sensitivity
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Sharpe ratio
axes[0, 0].plot(lookback_df['lookback'], lookback_df['sharpe'], marker='o', linewidth=2)
axes[0, 0].set_xlabel('Lookback (days)')
axes[0, 0].set_ylabel('Sharpe Ratio')
axes[0, 0].set_title('Sharpe Ratio vs Lookback Window')
axes[0, 0].grid(True, alpha=0.3)

# Annualized return
axes[0, 1].plot(lookback_df['lookback'], lookback_df['return'] * 100, 
               marker='o', linewidth=2, color='green')
axes[0, 1].set_xlabel('Lookback (days)')
axes[0, 1].set_ylabel('Return (%)')
axes[0, 1].set_title('Annualized Return vs Lookback Window')
axes[0, 1].grid(True, alpha=0.3)

# Volatility
axes[1, 0].plot(lookback_df['lookback'], lookback_df['volatility'] * 100, 
               marker='o', linewidth=2, color='orange')
axes[1, 0].set_xlabel('Lookback (days)')
axes[1, 0].set_ylabel('Volatility (%)')
axes[1, 0].set_title('Volatility vs Lookback Window')
axes[1, 0].grid(True, alpha=0.3)

# Max drawdown
axes[1, 1].plot(lookback_df['lookback'], lookback_df['max_dd'] * 100, 
               marker='o', linewidth=2, color='red')
axes[1, 1].set_xlabel('Lookback (days)')
axes[1, 1].set_ylabel('Max Drawdown (%)')
axes[1, 1].set_title('Max Drawdown vs Lookback Window')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Test position size limits
print('\nTesting position size limits...')

position_limits = [0.05, 0.10, 0.15, 0.20, 0.25]
position_results = []

for limit in position_limits:
    print(f"Testing {limit:.0%} position limit...")
    
    test_config = config.copy()
    test_config['portfolio']['max_position_size'] = limit
    
    test_portfolio_constructor = PortfolioConstructor(test_config)
    test_portfolios = test_portfolio_constructor.generate_portfolio_timeseries(
        signals_history, volatility, rebalance_dates.tolist()
    )
    
    test_backtester = CongressBacktester(test_config)
    test_results = test_backtester.simulate_portfolio(test_portfolios, prices)
    test_metrics = test_backtester.calculate_performance_metrics(test_results)
    
    position_results.append({
        'limit': limit,
        'sharpe': test_metrics['sharpe_ratio'],
        'return': test_metrics['annualized_return'],
        'max_dd': test_metrics['max_drawdown']
    })

position_df = pd.DataFrame(position_results)
print('\nPosition Limit Sensitivity:')
print(position_df.round(3))

## 4. Signal Decay Analysis

In [ ]:
# Analyze how signal predictive power decays over time
print('Signal Decay Analysis')
print('='*60)

# Calculate forward returns at different horizons
horizons = [1, 5, 10, 20, 40, 60]  # days

# Sample a subset of signals for analysis
sample_signals = []
for date, signals in list(signals_history.items())[::10]:  # Sample every 10th
    for _, signal in signals.iterrows():
        ticker = signal['ticker']
        if ticker in prices.columns:
            # Get forward returns
            try:
                current_price = prices.loc[date, ticker]
                
                for horizon in horizons:
                    future_date = date + pd.Timedelta(days=horizon)
                    if future_date in prices.index:
                        future_price = prices.loc[future_date, ticker]
                        forward_return = (future_price - current_price) / current_price
                        
                        sample_signals.append({
                            'signal': signal['signal_weighted'],
                            'horizon': horizon,
                            'forward_return': forward_return
                        })
            except:
                pass

signal_decay_df = pd.DataFrame(sample_signals)

# Calculate correlation by horizon
correlations = []
for horizon in horizons:
    horizon_data = signal_decay_df[signal_decay_df['horizon'] == horizon]
    if len(horizon_data) > 0:
        corr = horizon_data['signal'].corr(horizon_data['forward_return'])
        correlations.append({'horizon': horizon, 'correlation': corr})

corr_df = pd.DataFrame(correlations)

print('\nSignal-Return Correlation by Horizon:')
print(corr_df)

# Plot
plt.figure(figsize=(12, 6))
plt.plot(corr_df['horizon'], corr_df['correlation'], marker='o', linewidth=2)
plt.axhline(y=0, color='black', linestyle='--', alpha=0.3)
plt.xlabel('Horizon (Days)')
plt.ylabel('Correlation')
plt.title('Signal Decay: Correlation with Forward Returns')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Regulatory and Ethical Considerations

In [ ]:
# Display regulatory risk summary
print('REGULATORY AND ETHICAL RISK ASSESSMENT')
print('='*60)
print()
print('1. INFORMATION LAG:')
print('   - Congressional trades disclosed 30-45 days after execution')
print('   - Strategy acts on public information only')
print('   - No access to material non-public information')
print()
print('2. LEGAL COMPLIANCE:')
print('   - Uses publicly disclosed financial disclosures')
print('   - No insider trading concerns (public data)')
print('   - Complies with securities regulations')
print()
print('3. REPUTATIONAL RISK:')
print('   - Strategy may be viewed as exploiting political information')
print('   - Potential criticism for mimicking Congressional trades')
print('   - Consider ESG and ethical investment guidelines')
print()
print('4. CAPACITY CONSTRAINTS:')
print(f'   - Estimated capacity: ${capacity_estimate:,.0f}')
print(f'   - Strategy capacity utilization: {config["backtest"]["initial_capital"] / capacity_estimate:.1%}')
print('   - Limited scalability due to Congressional trade volumes')
print()
print('5. SIGNAL DETERIORATION RISK:')
print('   - As strategy becomes known, signal may deteriorate')
print('   - Crowding could reduce alpha')
print('   - Regular monitoring of signal quality required')
print()
print('RECOMMENDATION:')
print('- Implement as alternative/satellite strategy (<10% of portfolio)')
print('- Monitor regulatory changes to disclosure requirements')
print('- Regular review of ethical and compliance considerations')
print('- Maintain transparency with investors about strategy mechanics')

## Summary

Risk and capacity analysis reveals:
- Strategy capacity constraints due to Congressional trade volumes
- Optimal parameter ranges for lookback and position sizing
- Signal decay characteristics over different horizons
- Regulatory, reputational, and ethical considerations
- Recommendations for prudent implementation